# Diagnosing a Syslog file

This notebook uses an LLM to interpret a set of syslog entries.

In [ ]:
%load_ext autoreload
%autoreload 2

## Load Syslog Features
For the purposes of this experiment, we start with a JSON file capturing a collection of syslog entries, as features, grouped by source device for a specific issue we want to diagnose.  These features have been ranked as the most important log entries for performing a diagnosis.

In [ ]:
import os
import json

syslog_features = json.load(open('./datasets/syslog-demo.json'))
print(json.dumps(syslog_features, indent=2))

## Initialise LLM Client

In [ ]:
from modules.llm.azure_ai import AzureLlm
from modules.logger import Logger
from dotenv import load_dotenv

load_dotenv("env")

logger = Logger()
llm = AzureLlm(logger,os.getenv('AZURE_OPENAI_API_KEY'))

## Perform Initial Diagnosis
Use a prompt engineered to perform an initial draft diagnosis from the features (line entries) extracted from the device syslog file correlating the issue.

The syslog feature metadata indicates that the featured log entries were captured from a NETWORK-DEVICE. 

### Prompt Engineering: Network Diagnosis Prompt

Implement a specific prompt engineered to request the LLM to perform a network level diagnosis based upon the extracted syslog feature.


In [ ]:
from modules.llm.prompt import DiagnosisPrompt
from modules.llm.llm import LlmInterface
    
class SyslogDiagnosis(DiagnosisPrompt):
    _task_is_issue = """Below, in the 'syslog capture' section you will find a capture of the most important log entries from the specified 'syslog file name'.

Perform the following steps:

1) Determine whether the "syslog capture" is an issue containing anomalous information that could point to a potential problem, or whether the "syslog capture" just shows a system operating normally. DO NOT consider "SSH RSA Key Size compliance violation" as an issue.
2) If you determine the system is operating normally, then set 'issue' to false your response and do not perform steps 3 below (see "Example 1" below), otherwise continue on to next 3.
3) Create a technical 'description' of what this issue is (see "Example 2" below). Be sure to keep technical details in the description (such as host names, IP addresses, interfaces, ...). Also consider the "syslog file name" to determine the host name. 

Your response must only contain RFC8259 compliant JSON in the following format wrapped within [JSON] tags.
### Example 1:
[JSON]
{
    "issue": false
}
[JSON]

### Example 2:
[JSON]
{
    "issue": true,
    "description": "A technical description of the issue",
}
[JSON]
"""
    _schema_is_issue = _schema = {
        "type": "object",
        "properties": {
            "issue": {"type": "boolean"},
            "description": {"type": "string"}
        },
        "required": [
            "issue",            
        ]
    }

    _task_resolution = """Below, in the 'syslog capture' section you will find a capture of the most important log entries used to create the provided 'issue description'.

Perform the following steps:

1) Create a 'resolution', composed of a single paragraph detailing your suggested next steps to fix the provided 'issue description' for the 'syslog capture'. Be technical and specific.

Your response must only contain RFC8259 compliant JSON in the following format wrapped within [JSON] tags, see 'Example' below
### Example:
[JSON]
{
    "resolution: "Suggested next steps to fix the issue"
}
[JSON]
"""
    _schema_resolution = _schema = {
        "type": "object",
        "properties": {
            "resolution": {"type": "string"}
        },
        "required": [
            "resolution",            
        ]
    }
    
    _feature_desc = "Syslog capture"
    
    def __init__(self, logger, llm : LlmInterface, context):
        super().__init__(logger, llm, context=context, task=self._task_is_issue, feature_desc=self._feature_desc)
        
    def taskIsIssue(self, file, features):
        self.reset()
        self._schema = self._schema_is_issue
        self._task = self._task_is_issue
        super().run(features + "\n\n### syslog file name: " + file)                
        
    def taskResolution(self, features):
        if self.isIssue():
            self._schema = self._schema_resolution
            self._task = self._task_resolution
            super().run(features + "\n\n### issue description: " + self.getIssue())
            
    def run(self, file, features):
        self.taskIsIssue(file, features)
        self.taskResolution(features)
        
        
class NetworkDiagnosisPrompt(SyslogDiagnosis):
    _context = """You are a networking expert.
You are diagnosing a network issue based on syslog received from a Cisco router running IOSv 15.9."""
    
    def __init__(self, logger, llm : LlmInterface):
        super().__init__(logger, llm, self._context)
                        
    def run(self, file, features):
        self._logger.info('LLM Prompt: Diagnose Network Syslog features')
        super().run(file, features)

class KurbernetesPodDiagnosisPrompt(SyslogDiagnosis):
    _context = """You are an IT expert.
You are diagnosing a issue based on a syslog received from a Kubernetes pod."""
   
    def __init__(self, logger, llm : LlmInterface):
        super().__init__(logger, llm, self._context)
                        
    def run(self, file, features):
        self._logger.info('LLM Prompt: Diagnose Kubernetes Syslog features')
        super().run(file, features) 
        
class GenericDiagnosisPrompt(SyslogDiagnosis):
    _context = """You are an IT expert.
You are diagnosing an issue based on a syslog received."""
        
    def __init__(self, logger, llm : LlmInterface):
        super().__init__(logger, llm, self._context)
                        
    def run(self, file, features):
        self._logger.info('LLM Prompt: Diagnose Syslog features')
        super().run(file, features)

### Create the Initial Diagnosis
Use the featured device type to instantiate and run the Network Diagnosis prompt.


In [ ]:
syslog_diagnoses = []

for syslog_feature in syslog_features:
    # Create a dictionary to capture the details of the syslog diagnosis
    syslog_diagnosis = {
        'Entity': syslog_feature['Entity'],
        'Features': '\n'.join(str(s) for s in syslog_feature["Features"]),
        'Issue': False,
        'Description': "NONE",
        'Resolution': "NONE"
    }
    
    diagnosisPrompt = None
    match syslog_feature["Type"]:
        case "NETWORK_DEVICE":
            diagnosisPrompt = NetworkDiagnosisPrompt(logger, llm)
        case "KUBERNETES_POD":
            diagnosisPrompt = KurbernetesPodDiagnosisPrompt(logger, llm)
        case _:
            diagnosisPrompt = GenericDiagnosisPrompt(logger, llm)
        
    
    # First task is to get the LLM to determine whether the extracted syslog features correlate to an issue of a potential problem that needs to be resolved.
    diagnosisPrompt.taskIsIssue(syslog_feature['File'], syslog_diagnosis['Features'])
    syslog_diagnosis['Issue'] = diagnosisPrompt.isIssue()
    
    if diagnosisPrompt.isIssue():
        syslog_diagnosis['Description'] = diagnosisPrompt.getIssue()
        diagnosisPrompt.taskResolution(syslog_diagnosis['Features'])
        syslog_diagnosis['Resolution'] = diagnosisPrompt.getResolution()
                
    syslog_diagnoses.append(syslog_diagnosis)



In [ ]:
import modules.utils as utils
utils.displayDictionary(syslog_diagnoses)